# Table annotations

In [1]:
from rich.console import Console
from rich.panel import Panel

console = Console(
    width=180,  # for getting Markdown tables rendered nicely
)

In [2]:
from docling_core.types.doc.document import DoclingDocument

file_path = "2408.09869v3.json"
pages = {5}  # pages to serialize (for output brevity)

doc = DoclingDocument.load_from_json(file_path)

## Adding a table annotation

Below we add a demo table annotation, picking the first table for illustrative purposes.

Note that `TableMiscData` allows any dict data within the `content` field.


In [3]:
from docling_core.types.doc.document import TableMiscData

assert doc.tables, "No table available in this document"
table = doc.tables[0]

my_annotation = TableMiscData(
    content={
        "summary": "Typical Docling setup runtime characterization.",
        "type": "performance data",
        # ...
    },
)
table.add_annotation(
    annotation=my_annotation,
)

## Default serialization

In [4]:
from docling_core.transforms.serializer.markdown import (
    MarkdownDocSerializer,
    MarkdownParams,
)

ser = MarkdownDocSerializer(
    doc=doc,
    params=MarkdownParams(
        pages=pages,
    ),
)
ser_out = ser.serialize()
ser_txt = ser_out.text
console.print(Panel(f"{ser_txt[:2000]}...", title=f"{pages=}"))

╭─────────────────────────────────────────────────────────────────────────────────── pages={5} ────────────────────────────────────────────────────────────────────────────────────╮
│ torch runtimes backing the Docling pipeline. We will deliver updates on this topic at in a future version of this report.                                                        │
│                                                                                                                                                                                  │
│ Table 1: Runtime characteristics of Docling with the standard model pipeline and settings, on our test dataset of 225 pages, on two different systems. OCR is disabled. We show  │
│ the time-to-solution (TTS), computed throughput in pages per second, and the peak memory used (resident set size) for both the Docling-native PDF backend and for the pypdfium   │
│ backend, using 4 and 16 threads.                                                                                                                                                 │
│                                                                                                                                                                                  │
│ | CPU                              | Thread budget   | native backend   | native backend   | native backend   | pypdfium backend   | pypdfium backend   | pypdfium backend   |   │
│ |----------------------------------|-----------------|------------------|------------------|------------------|--------------------|--------------------|--------------------|   │
│ |                                  |                 | TTS              | Pages/s          | Mem              | TTS                | Pages/s            | Mem                |   │
│ | Apple M3 Max                     | 4               | 177 s 167 s      | 1.27 1.34        | 6.20 GB          | 103 s 92 s         | 2.18 2.45          | 2.56 GB            |   │
│ | (16 cores) Intel(R) Xeon E5-2690 | 16 4 16         | 375 s 244 s      | 0.60 0.92        | 6.16 GB          | 239 s 143 s        | 0.94 1.57          | 2.42 GB            |   │
│                                                                                                                                                                                  │
│ ## 5 Applications                                                                                                                                                                │
│                                                                                                                                                                                  │
│ Thanks to the high-quality, richly structured document conversion achieved by Docling, its output qualifies for numerous downstream applications. For example, Docling can       │
│ provide a base for detailed enterprise document search, passage retrieval or classification use-cases, or support knowledge extraction pipelines, allowing specific treatment of │
│ different structures in the document, such as tables, figures, section structure or references. For popular generative AI application patterns, such as retrieval-augmented      │
│ generation (RAG), we provide quackling , an open-source package which capital...                                                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Custom serialization

In [5]:
from typing import Any

from docling_core.transforms.serializer.base import (
    BaseDocSerializer,
    SerializationResult,
)
from docling_core.transforms.serializer.common import create_ser_result
from docling_core.transforms.serializer.markdown import MarkdownTableSerializer
from docling_core.types.doc.document import (
    TableItem,
)
from typing_extensions import override


class CustomAnnotationTableSerializer(MarkdownTableSerializer):
    @override
    def serialize(
        self,
        *,
        item: TableItem,
        doc_serializer: BaseDocSerializer,
        doc: DoclingDocument,
        **kwargs: Any,
    ) -> SerializationResult:
        params = MarkdownParams(**kwargs)

        text_parts: list[str] = []

        if params.include_annotations:
            for ann in item.annotations:
                if isinstance(ann, TableMiscData):

                    # custom serialization logic:
                    out_txt = "\n".join([f"{k}: {ann.content[k]}" for k in ann.content])
                    text_parts.append(out_txt)

        # reusing the existing result (excluding the annotations):
        parent_res = super().serialize(
            item=item,
            doc_serializer=doc_serializer,
            doc=doc,
            **{**kwargs, **{"include_annotations": False}},
        )
        text_parts.append(parent_res.text)

        text_res = "\n\n".join(text_parts)
        return create_ser_result(text=text_res, span_source=item)

In [6]:
ser = MarkdownDocSerializer(
    doc=doc,
    table_serializer=CustomAnnotationTableSerializer(),
    params=MarkdownParams(
        pages=pages,
    ),
)
ser_out = ser.serialize()
ser_txt = ser_out.text
console.print(Panel(f"{ser_txt[:2000]}...", title=f"{pages=}"))

╭─────────────────────────────────────────────────────────────────────────────────── pages={5} ────────────────────────────────────────────────────────────────────────────────────╮
│ torch runtimes backing the Docling pipeline. We will deliver updates on this topic at in a future version of this report.                                                        │
│                                                                                                                                                                                  │
│ summary: Typical Docling setup runtime characterization.                                                                                                                         │
│ type: performance data                                                                                                                                                           │
│                                                                                                                                                                                  │
│ Table 1: Runtime characteristics of Docling with the standard model pipeline and settings, on our test dataset of 225 pages, on two different systems. OCR is disabled. We show  │
│ the time-to-solution (TTS), computed throughput in pages per second, and the peak memory used (resident set size) for both the Docling-native PDF backend and for the pypdfium   │
│ backend, using 4 and 16 threads.                                                                                                                                                 │
│                                                                                                                                                                                  │
│ | CPU                              | Thread budget   | native backend   | native backend   | native backend   | pypdfium backend   | pypdfium backend   | pypdfium backend   |   │
│ |----------------------------------|-----------------|------------------|------------------|------------------|--------------------|--------------------|--------------------|   │
│ |                                  |                 | TTS              | Pages/s          | Mem              | TTS                | Pages/s            | Mem                |   │
│ | Apple M3 Max                     | 4               | 177 s 167 s      | 1.27 1.34        | 6.20 GB          | 103 s 92 s         | 2.18 2.45          | 2.56 GB            |   │
│ | (16 cores) Intel(R) Xeon E5-2690 | 16 4 16         | 375 s 244 s      | 0.60 0.92        | 6.16 GB          | 239 s 143 s        | 0.94 1.57          | 2.42 GB            |   │
│                                                                                                                                                                                  │
│ ## 5 Applications                                                                                                                                                                │
│                                                                                                                                                                                  │
│ Thanks to the high-quality, richly structured document conversion achieved by Docling, its output qualifies for numerous downstream applications. For example, Docling can       │
│ provide a base for detailed enterprise document search, passage retrieval or classification use-cases, or support knowledge extraction pipelines, allowing specific treatment of │
│ different structures in the document, such as tables, figures, section structure or references. For popular generative AI application patterns, such as retrieval-augmen...      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯